In this project, we will build a regression (predictive) model using Keras to model the data (create a neural network and build & train a model) about concrete compressive strength. The data can be found at: https://cocl.us/concrete_data.

Let's start by importing the related libraries as needed.

In [2]:
# import the following libraries
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


### Part A

Let's download the data and read it into a <em>pandas</em> dataframe.

In [3]:
concrete_dat = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_dat.tail(4)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77
1029,260.9,100.5,78.3,200.6,8.6,864.5,761.5,28,32.40


Let's explore more about the dataset.

In [4]:
concrete_dat.shape

(1030, 9)

We'll randomly split the data, so there are less than 1000 samples to train our model on - at this point, we have to be careful not to overfit the training data.

In [5]:
concrete_dat.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


Before data-manipulation, preprocessing or feature-engineering, we might also want to find out the relationship between variables.

In [6]:
# use Pearson method
concrete_dat.corr(method ='pearson') 

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
Cement,1.000000,-0.275216,-0.397467,-0.081587,0.092386,-0.109349,-0.222718,0.081946,0.497832
Blast Furnace Slag,-0.275216,1.000000,-0.323580,0.107252,0.043270,-0.283999,-0.281603,-0.044246,0.134829
Fly Ash,-0.397467,-0.323580,1.000000,-0.256984,0.377503,-0.009961,0.079108,-0.154371,-0.105755
Water,-0.081587,0.107252,-0.256984,1.000000,-0.657533,-0.182294,-0.450661,0.277618,-0.289633
Superplasticizer,0.092386,0.043270,0.377503,-0.657533,1.000000,-0.265999,0.222691,-0.192700,0.366079
Coarse Aggregate,-0.109349,-0.283999,-0.009961,-0.182294,-0.265999,1.000000,-0.178481,-0.003016,-0.164935
Fine Aggregate,-0.222718,-0.281603,0.079108,-0.450661,0.222691,-0.178481,1.000000,-0.156095,-0.167241
Age,0.081946,-0.044246,-0.154371,0.277618,-0.192700,-0.003016,-0.156095,1.000000,0.328873
Strength,0.497832,0.134829,-0.105755,-0.289633,0.366079,-0.164935,-0.167241,0.328873,1.000000


Let's check the dataset for any missing values.

In [7]:
concrete_dat.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks pretty much ready...

So let's split data into predictors and target. The target variable in this problem is the concrete sample strength.

In [8]:
dat_cols = concrete_dat.columns

predictors = concrete_dat[dat_cols[dat_cols != 'Strength']]  # all columns except Strength
target = concrete_dat['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.

In [9]:
predictors[:4]

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365


In [12]:
target[:4]

0    79.99
1    61.89
2    40.27
3    41.05
Name: Strength, dtype: float64

Now we can save the number of predictors into *n_cols* as we need this number later for building our neural network.

In [14]:
n_cols = predictors.shape[1]   # number of predictors

And, let's randomly split the data: holding 30% of the data for testing.

In [15]:
# split the data: holding 30% of the data for testing

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)

#### We now start building a Neural Network

First, let's define our regression model by creating a function so that we're able to conveniently call it during our model creation.

In [16]:
# define a regression model: one hidden layer of 10 nodes and use ReLU activation function

def regression_model():

    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

#### Train and Test the Network

Let's call the function now to create our model.

In [17]:
# build the model
model = regression_model()

Here we will train and test the model at the same time using the *fit* method. We will train the model for 50 epochs at this time.

In [18]:
#####################
# model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)
# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=1)

Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 [==============================] - 0s 247us/step - loss: 241623.2450 - val_loss: 170624.6457
Epoch 2/50
721/721 [==============================] - 0s 30us/step - loss: 124799.6619 - val_loss: 82735.7879
Epoch 3/50
721/721 [==============================] - 0s 31us/step - loss: 59266.4731 - val_loss: 37432.7567
Epoch 4/50
721/721 [==============================] - 0s 29us/step - loss: 27086.8555 - val_loss: 16751.1701
Epoch 5/50
721/721 [==============================] - 0s 31us/step - loss: 12820.1982 - val_loss: 8626.2377
Epoch 6/50
721/721 [==============================] - 0s 29us/step - loss: 7447.6260 - val_loss: 5740.3743
Epoch 7/50
721/721 [==============================] - 0s 31us/step - loss: 5587.9057 - val_loss: 4768.3054
Epoch 8/50
721/721 [==============================] - 0s 30us/step - loss: 4937.9253 - val_loss: 4397.0368
Epoch 9/50
721/721 [==============================] - 0s 29us/step - loss: 4621.2290 

In [31]:
# evaluate the model

# pred= model.predict(X_test)
# print('RMSE:', np.sqrt(mean_squared_error(y_test,pred))) 

Let's put the steps together and train & test the model for a number of 50 times. To evaluate our model, we'll calculate the mean of RMSE and find out the standard deviaiton.

In [19]:
# randomly split the data & train the model for a number of 50 times

RMSE = []
for i in range(50):          
    
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=0)
    
    pred= model.predict(X_test)
    _output = np.sqrt(mean_squared_error(y_test,pred))
    
    RMSE.append(_output)

print(RMSE)
print('-------------Part A------------')
print('Mean of RMSE:', np.mean(RMSE))
print('Standard Deviation of RMSE:', np.std(RMSE))
print('Times of Run:', len(RMSE))

[13.641154063744153, 12.25310319114476, 14.130602586102704, 11.857081428994823, 14.936947170098792, 27.444370825409447, 10.022470015939813, 13.806232090962235, 24.898261294286968, 9.031729718856758, 25.995218089484407, 10.41135065861988, 49.59387677110834, 20.580652641159723, 10.049870049613, 10.665492056104634, 25.55445395124983, 13.495026274162553, 10.874420762640515, 10.23049698195622, 23.699969887250905, 11.663229528827111, 13.549683897461307, 22.73241934208476, 8.966649978310109, 10.319234439983047, 16.413227391986364, 22.032033457176098, 15.984186588487125, 11.123065331928574, 12.542217715157149, 14.730741593877633, 12.531061008464738, 11.032360800273178, 22.36289368649462, 10.496870850398986, 37.765527093532846, 18.899531718344857, 16.379359273876457, 11.60449720420003, 13.646975980221592, 15.61252359132778, 12.40179538277311, 13.391458037482202, 18.643708157155647, 24.019704736765675, 8.404825672636903, 30.61856724566394, 32.365388068654035, 11.708196195050308]
-------------Par

We got - Mean of RMSE: 16.902 & Standard Deviation of RMSE: 8.186

### Part B

At this part, we'll normalize the data by substracting the mean and dividing by the standard deviation.

In [20]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head(4)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221


In [21]:
# repeat the step as in Part A
n_cols = predictors_norm.shape[1] # number of predictors

In [22]:
# define a regression model: one hidden layer of 10 nodes and use ReLU activation function

def regression_model2():

    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Similarly, we put the steps together and train & test the model for a number of 50 times but using normalized data at this time. To evaluate our model, we'll calculate the mean of RMSE.

In [23]:
RMSE = []
for i in range(50):          
    
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    model = regression_model2()
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=0)
    
    pred= model.predict(X_test)
    _output = np.sqrt(mean_squared_error(y_test,pred))
    
    RMSE.append(_output)

print(RMSE)
print('-------------Part B------------')
print('Mean of RMSE:', np.mean(RMSE))
# print('Standard Deviation of RMSE:', np.std(RMSE))
print('Times of Run:', len(RMSE))

[18.583063927681096, 17.135096526039714, 18.67859642791842, 21.017707627140005, 23.286909976812645, 18.98258082231753, 16.905363627218705, 23.301660842938315, 18.46122183713074, 22.33059557530496, 17.4926427877145, 19.860766847327174, 17.688842689669467, 18.400883594234937, 15.612978421290663, 16.162932474709734, 24.512067861136156, 26.415069407201898, 17.89354041076253, 18.16284569504621, 17.457724525584304, 22.21290828205874, 21.644044426210478, 16.71810134729673, 20.021714262210484, 29.4602081784468, 22.728428235838354, 21.50610131763676, 15.965612353549005, 20.082865306723214, 17.55157233741699, 17.747125081192085, 15.981399247335546, 18.897274412859886, 18.193211680423534, 19.131792364095897, 17.625658045461094, 17.474962398169918, 17.30646450424493, 16.22496410049844, 18.132431319513234, 19.442400857951306, 20.17685842719881, 26.645157699177087, 23.00079897474266, 18.662568183374294, 16.460494464132026, 25.414920997544154, 19.379692376744064, 22.194587987946832]
-------------Part

Compared to 16.902 in Part A, we have the mean of RMSE: 19.687. We can see the mean slightly going up but still not significantly changed when normalizing the data.

### Part C

At this part, we use 100 epochs for training (& testing) at this time and see what is the output for the mean of RMSE.

In [24]:
RMSE = []
for i in range(50):          
    
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    model = regression_model2()
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=0)
    
    pred= model.predict(X_test)
    _output = np.sqrt(mean_squared_error(y_test,pred))
    
    RMSE.append(_output)

print(RMSE)
print('-------------Part C------------')
print('Mean of RMSE:', np.mean(RMSE))
# print('Standard Deviation of RMSE:', np.std(RMSE))
print('Times of Run:', len(RMSE))

[13.27464959183036, 12.645953739327657, 13.409508235977071, 12.376877517827266, 11.6894017446621, 12.232659506376006, 15.517954609442363, 13.400551477682582, 13.504539654149944, 13.075969978492695, 12.829706096089353, 13.835160973462855, 12.75838982472191, 13.172105669216934, 12.159682366577877, 12.48740490476878, 13.777131796988995, 15.206055258860461, 12.696301954310071, 12.250726211623588, 13.303453178730996, 13.590993976997542, 12.600752123098927, 12.44858737216601, 11.849640959274314, 13.002734129567004, 12.994392672098856, 13.579878399060648, 13.287437320453561, 12.243643802804367, 12.970121810531268, 12.656488641119914, 11.910620009130927, 14.590310678150562, 11.865435324670065, 13.625248444421086, 12.180042791749099, 13.656398593201086, 12.3330052540298, 12.014729328212558, 11.929166734795606, 12.896000151260644, 12.611554672043995, 11.99604941590147, 12.447450547607746, 12.632954963191622, 12.313516135636707, 13.141936339428106, 11.696248487042471, 13.95139491782283]
---------

Compared to 19.687 in Part B, we have the mean of RMSE: 12.892. We can see - increasing the epoch number (which means increasing the passes for forward & backward in the neural network), we have smaller RMSE (the better result).

### Part D

At this part, we define a new model: 3 hidden layers, each of 10 nodes and use ReLU activation function and see what is the output for the mean of RMSE. 

In [25]:
# define to a new regression model 

def rgres_model3():

    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Similarly to Part B, we put the steps together and train & test the model for a number of 50 times. To evaluate our model, we'll calculate the mean of RMSE.

In [26]:
RMSE = []
for i in range(50):          
    
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    model = rgres_model3()
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=0)
    
    pred= model.predict(X_test)
    _output = np.sqrt(mean_squared_error(y_test,pred))
    
    RMSE.append(_output)

print(RMSE)
print('-------------Part D------------')
print('Mean of RMSE:', np.mean(RMSE))
# print('Standard Deviation of RMSE:', np.std(RMSE))
print('Times of iteration:', len(RMSE))

[10.407370534957536, 9.212553748387368, 11.931190577813592, 11.394610997886032, 11.433923142757159, 11.440847660568897, 11.684091118175228, 11.374396055025857, 9.429073270072289, 12.251710199937486, 11.577341967750563, 11.292803931348226, 12.070573035310197, 11.680254381131196, 11.226069182784071, 11.496427824580845, 11.76491409543739, 11.136083900232464, 10.810217965081003, 12.794620841300377, 11.855900655954672, 12.345744687171168, 11.563518282815277, 12.077923652479706, 11.37035138392548, 11.581425730470086, 11.59505023533354, 10.860734675729033, 11.184037811130477, 12.90657386792141, 11.685762903318743, 10.192668056994865, 11.40359668090261, 10.513988837813319, 10.335056332948618, 10.59640289008547, 11.096848657089668, 11.815788042874566, 12.777439534919775, 11.862600160631642, 11.592213871947026, 10.025808651571602, 10.11969735016051, 12.185777908999452, 11.805133544899238, 12.508485763465961, 10.734685358788754, 10.80874500715579, 11.144941063767815, 12.114138919054739]
---------

Compared to 19.687 in Part B, we have the mean of RMSE: 11.381. We can see - This was the smallest RMSE when compared to the values in the previous parts - By increasing the epoch number and the network layers, we can see an improvement of RMSE. 

The output above (Part D) shows that the RMSE, which is our evaluation metric, was 11.381 - Ideally, the lower the RMSE value, the better the model performance. But - Keep that in mind, in contrast to accuracy, it is not straightforward to interpret RMSE. 

Thank you!